In [56]:
import pya
import numpy as np

In [179]:
def merge_polygons(polys):
    """Merges the shapes in polys into one polygon. Overlaps will be merged into one.
    
    @param list polys: list of the polygons that should be merged.

    @return poly.Region: region containing the merged polygon
    """
    union = pya.Region()
    for poly in polys:
        union += pya.Region(poly)
    merged = union.merge()
    return(merged)


def merge_regions(regions):
    """Merges the regions in regions into one region. Overlaps will be merged into one?
    
    @param list regions: list of the regions that should be merged.

    @return Region: region containing the merged regions
    """
    union = pya.Region()
    for region in regions:
        union += region
    merged = union.merge()
    return(merged)


def create_box(center, width, height):
    """Creates a box.
    
    @param list center: x and y coordinate of the center of the box

    @param float width: width (x) of the box

    @param float height: height (y) of the box

    @return Box: Box object
    """
    box = pya.Box.new(pya.Point(center[0]-width/2,center[1]-height/2),pya.Point(center[0]+width/2,center[1]+height/2))
    return(box)


def circle(center,radius,n_points):
    """returns points that form a circle
    
    @param int n_points:
    
    @param float radius:
    
    @param list center:
    """
    pts = []
    for i in range(0,n_points+1): # first and last point are identical
        angle = 2*np.pi*i/n_points
        x = center[0] + radius*np.cos(angle)
        y = center[1] + radius*np.sin(angle)
        pts.append((x,y))

    return(pts)


def create_circle(center,radius,n_points):
    structure = []
    for i in circle(center,radius,n_points):
        structure.append(pya.Point.new(i[0],i[1]))

    poly = pya.Polygon.new(structure)
    return(poly)


def create_semicircle(center,r1,r2,n_points):
    """r2>r1"""
    circle1 = create_circle(center,r1,n_points)
    circle2 = create_circle(center,r2,n_points)
    semi = pya.Region(circle2) - pya.Region(circle1)
    semi = semi - pya.Region(create_box([center[0],center[1]-r2/2],2*r2,r2))

    return(semi)


def create_text(posi,gridsize,height,text,anchor='bottomleft'): #text,gridsize,height,posi,anchor='bottomleft'):
    """Creates a text object at the specified location.
    
    @param str text: ext to display

    @param float gridsize: size of the used grid (layout.dbu)

    @param float height: height of the text

    @param list posi: x and y coordinate of the box (lower left corner)
    """
    scale = height * 10/7 * 1e-3 # rescaling so size is in dbu
    region = pya.TextGenerator.default_generator().text(text,gridsize/scale)
    txtheight = region.bbox().height()
    txtwidth = region.bbox().width()
    if anchor == 'bottomleft':
        region.move(int(posi[0]), int(posi[1]))
    elif anchor == 'center':
        region.move(int(posi[0])-txtwidth/2, int(posi[1])-txtheight/2)
    else: 
        print('anchor unknown, using bottomleft.')
    return(region)


def create_marker(center,width):
    """Creates a marker that is centered at center and width wide and tall.
    """
    boxlength = 0.3*width
    boxwidth = 0.1*width
    connectorwidth = 0.01*width
    arms_offset = 0.5*width-0.5*boxlength

    elements = []
    elements.append(create_box(center, width, connectorwidth)) # connector_vertical
    elements.append(create_box(center, connectorwidth, width)) # connector_horizontal
    elements.append(create_box([center[0],center[1]+arms_offset],boxwidth,boxlength)) # upper arm
    elements.append(create_box([center[0],center[1]-arms_offset],boxwidth,boxlength)) # lower arm
    elements.append(create_box([center[0]-arms_offset,center[1]],boxlength,boxwidth)) # left arm
    elements.append(create_box([center[0]+arms_offset,center[1]],boxlength,boxwidth)) # right arm

    marker = merge_polygons(elements)

    return(marker)


def create_marker_matrix(center_marker1,width_marker,repetitions,offsets):
    """ Creates an array of markers
    
    @param list center_marker1: list of floats that gives the position (x and y) of the marker in the lower left corner of the array

    @param float width_marker: size of the marker

    @param list repetitions: list of ints with the number of repetitions in x and y direction

    @param list offsets: list of floats that gives the offset between markers in x and y direction

    @return Region: region that contains the markers
    """

    markers = []
    center = [0,0] # initialize center
    for i in range(repetitions[0]):
        center[0] = center_marker1[0] + i*offsets[0]
        for j in range(repetitions[1]):
            center[1] = center_marker1[1] + j*offsets[1]
            marker = create_marker(center,width_marker)
            markers.append(marker)
    matrix = merge_regions(markers)
    
    return(matrix)


def create_numbers_for_marker(center,gridsize,crosswidth,numbers):
    """Creates 4 numbers in a square.

    If set up correctly, numbers should be in quadrants of the corresponding marker/cross.

    @param list center: x and y coordinate of the crosses center

    @param float gridsize: size of hte used grid (layout.dbu)

    @param float crosswidth: width of the corresponding cross

    @param list numbers: numbers as string that should be placed in the quadrants.
        The order is a follows: [top left, top right, lower left, lower right]
    """
    textheight = 0.3*crosswidth
    elements = []
    elements.append(create_text([center[0]-crosswidth/4,center[1]+crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[0],
                                anchor = 'center')
                    ) # top left
    elements.append(create_text([center[0]+crosswidth/4,center[1]+crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[1],
                                anchor = 'center')
                    ) # top right
    elements.append(create_text([center[0]-crosswidth/4,center[1]-crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[2],
                                anchor = 'center')
                    ) # bottom left
    elements.append(create_text([center[0]+crosswidth/4,center[1]-crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[3],
                                anchor = 'center')
                    ) # bottom right

    numbers = merge_regions(elements)

    return numbers


def create_number_matrix(center_marker1,gridsize,width_marker,repetitions,offsets):
    """ Creates an array of numbers.
    
    @param list center_marker1: list of floats that gives the position (x and y) of the corresponding marker in the lower left corner of the array

    @param float width_marker: size of the corresponding marker

    @param list repetitions: list of ints with the number of repetitions in x and y direction

    @param list offsets: list of floats that gives the offset between corresponding markers in x and y direction

    @return Region: region that contains the numbers
    """
    elements = []
    center = [0,0] # initialize center
    for i in range(repetitions[0]):
        center[0] = center_marker1[0] + i*offsets[0]
        num_x = f'{i:02}'
        for j in range(repetitions[1]):
            center[1] = center_marker1[1] + j*offsets[1]
            num_y = f'{j:02}'
            numbers = [num_x[:-1],num_x[-1],num_y[:-1],num_y[-1]] # takes last number in right quadrants, rest (usually only 1) in left quadrants
            nums = create_numbers_for_marker(center,gridsize,width_marker,numbers) # TODO: check order fo arguments here
            elements.append(nums)
    matrix = merge_regions(elements)

    return matrix


def create_smiley_coupler(center,length,height,r1,r2,r3,r4,n_points):
    """TODO: make more nice"""
    outside = create_box(center,length,height)
    center,r1,r2,n_points
    semi1 = create_semicircle([center[0],center[1]-height/2+200],r1,r2,n_points)
    semi2 = create_semicircle([center[0],center[1]-height/2+200],r3,r4,n_points)
    smiley = pya.Region(outside) - semi1 - semi2

    return(smiley)

In [180]:
## create new layout
layout = pya.Layout()
layout.dbu = 1e-3 # data base unit in um --> each number we put is now in nm

top = layout.create_cell("TOP")
l1 = layout.layer(1, 0)
l2 = layout.layer(2,0)

## create structures
width_marker = 10000
center_marker1 = [width_marker/2,width_marker/2]
repetitions = [3,3]
offsets = [50000,50000]
marker_matrix = create_marker_matrix(center_marker1=center_marker1,
                                        width_marker=width_marker,
                                        repetitions=repetitions, 
                                        offsets=offsets)
# TODO: check order of arguments here
number_matrix = create_number_matrix(center_marker1=center_marker1,
                                        gridsize=layout.dbu,
                                        width_marker=width_marker,
                                        repetitions=repetitions,
                                        offsets=offsets)
semicircle = create_semicircle([0,0],5000,10000,20)
smiley = create_smiley_coupler(center=[0,0],length=4000,height=2500,r1=150,r2=520,r3=680,r4=1050,n_points=20)
## place structures
# top.shapes(l1).insert(marker_matrix)
# top.shapes(l2).insert(number_matrix)
# top.shapes(l1).insert(semicircle)
top.shapes(l1).insert(smiley)

## save in file
layout.write('t.gds')